In [20]:

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


pairs = [
    ("i love ai", "ai love i"),
    ("students learn models", "models learn students"),
    ("deep learning rocks", "rocks learning deep"),
    ("attention helps learning", "learning helps attention"),
    ("transformers are powerful", "powerful are transformers"),
]

input_texts = [p[0] for p in pairs]
target_texts = ["<start> " + p[1] + " <end>" for p in pairs]


In [21]:


# 2️. Tokenization

tokenizer_inputs = Tokenizer()
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)
num_encoder_tokens = len(tokenizer_inputs.word_index) + 1

tokenizer_outputs = Tokenizer(filters="")
tokenizer_outputs.fit_on_texts(target_texts)
target_sequences = tokenizer_outputs.texts_to_sequences(target_texts)
num_decoder_tokens = len(tokenizer_outputs.word_index) + 1

max_encoder_seq_length = max(len(seq) for seq in input_sequences)
max_decoder_seq_length = max(len(seq) for seq in target_sequences)


In [22]:


# 3️. Padding + one-hot targets

encoder_input_data = pad_sequences(input_sequences, maxlen=max_encoder_seq_length, padding="post")
decoder_input_data = pad_sequences(target_sequences, maxlen=max_decoder_seq_length, padding="post")

decoder_target_data = np.zeros((len(pairs), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
for i, seq in enumerate(target_sequences):
    for t, word_id in enumerate(seq[1:]):  # shift by 1
        decoder_target_data[i, t, word_id] = 1.0


In [23]:


# 4️. Seq2Seq model definition

latent_dim = 64
embed_dim = 50

# ----- Encoder -----
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, embed_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# ----- Decoder -----
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, embed_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# ----- Full training model -----
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [24]:


# 5️. Train

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=2,
    epochs=300,
    verbose=0,
)
print("Training complete!\n")


Training complete!



In [25]:


# 6️. Build inference models
# At inference, we reuse these same layers, but drive them manually one token at a time.

# Encoder
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder (single-step)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_single_input = Input(shape=(1,))
dec_emb2 = dec_emb_layer(decoder_single_input)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs
)
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_states2 = [state_h2, state_c2]

decoder_model = Model(
    [decoder_single_input] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2,
)


In [26]:

# 
# 7️. Decode helper
# 
reverse_target_index = {i: w for w, i in tokenizer_outputs.word_index.items()}

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer_outputs.word_index["<start>"]

    decoded_sentence = ""
    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_index.get(sampled_token_index, "")

        if sampled_word in ("<end>", ""):
            break

        decoded_sentence += " " + sampled_word
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()


In [27]:

# 
# 8️. Prediction utility
# 
def predict_new_sentence(sentence):
    seq = tokenizer_inputs.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_encoder_seq_length, padding="post")
    translation = decode_sequence(seq)
    print(f"Input: {sentence}")
    print(f"Predicted Output: {translation}\n")


In [28]:

# ------------------------------------------------
# 9️. Test predictions
# ------------------------------------------------
predict_new_sentence("i love ai")
predict_new_sentence("students learn models")
predict_new_sentence("deep learning rocks")
predict_new_sentence("attention helps learning")
predict_new_sentence("transformers are powerful")

# Completely new ones
predict_new_sentence("ai helps students")
predict_new_sentence("learning is fun")
predict_new_sentence("models learn quickly")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Input: i love ai
Predicted Output: ai love i

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Input: students learn models
Predicted Output: models learn students

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input: deep learning rocks
Predicted Output: rocks learning deep

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input: attention helps learning
Predicted Output: learnin